https://colab.research.google.com/github/dlsyscourse/public_notebooks/blob/main/transformer_implementation.ipynb


In [3]:
import numpy as np
import torch

`y = softmax(K Q^T / sqrt(d)) V`
`y = ( softmax(X W_K W_Q^T X) / sqrt(d)) X W_v ) W_out`

W_out seem to be uncessary since in above formular it can lum into W_v. But it will be useful in multi-head attn.

In [48]:
def softmax(Z):
    Z = np.exp(Z - Z.max(axis=-1, keepdims=True)) # always take the softmax of the last dim
    return Z / Z.sum(axis=-1, keepdims=True)

def self_attn(X, mask, W_KQV, W_out):
    K,Q,V = np.split(X @ W_KQV, 3, axis=-1)
    # print(K.shape,Q.shape,V.shape)
    bach_size, d = X.shape
    attn = softmax(K @ Q.swapaxes(-1, -2) / np.sqrt(d) + mask)
    # print(">>>", attn.shape, V.shape, W_out.shape)
    return attn @ V @ W_out, attn

In [49]:
T, d = 100, 64
attn = torch.nn.MultiheadAttention(d, 1, bias=False, batch_first=True) # 1 head
mask = torch.triu(-float("inf")*torch.ones(T,T), 1)
X = torch.randn(1,T,d)
Y_, A_ = attn(X,X,X, attn_mask=mask)
# print(mask, X, "\n")
# print(Y_, A_)

In [50]:
W_KQV = attn.in_proj_weight.detach().numpy().T
W_out = attn.out_proj.weight.detach().numpy().T

assert W_out.shape == (64, 64), W_out.shape
assert W_KQV.shape[1] == 64 * 3, W_KQV.shape # k, q, v = 64 + 64 + 64

Y, A = self_attn(X[0].numpy(), mask.numpy(), W_KQV, W_out)

In [51]:
print(np.linalg.norm(A - A_[0].detach().numpy()))
print(np.linalg.norm(Y - Y_[0].detach().numpy()))


2.395901e-07
1.6370427e-06
